In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend as K
import gc

# custom module for Capstone 2 project
import cap2tools as c2t

Using TensorFlow backend.


In [2]:
# configure GPU memory usage by tensorflow
config = K.tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.90
K.tensorflow_backend.set_session(K.tf.Session(config=config))

In [3]:
# run model evaluation
data_path = 'sampled_photos/val'

model_paths = {'0':'vgg16_dropout1_0_1.h5', 
               '0.1':'vgg16_dropout1_0-1_1.h5', 
               '0.2':'vgg16_dropout1_0-2_1.h5', 
               '0.3':'vgg16_dropout1_0-3_1.h5', 
               '0.4':'vgg16_dropout1_0-4_1.h5', 
               '0.5':'vgg16_dropout1_0-5_1.h5', 
               '0.6':'vgg16_dropout1_0-6_1.h5', 
               '0.7':'vgg16_dropout1_0-7_1.h5', 
               '0.8':'vgg16_dropout1_0-8_1.h5'}

model_metrics = c2t.eval_models(model_paths, data_path)

Building image generator...
Found 1000 images belonging to 5 classes.
Loading model vgg16_dropout1_0_1.h5
Evaluating model vgg16_dropout1_0_1.h5
Loading model vgg16_dropout1_0-1_1.h5
Evaluating model vgg16_dropout1_0-1_1.h5
Loading model vgg16_dropout1_0-2_1.h5
Evaluating model vgg16_dropout1_0-2_1.h5
Loading model vgg16_dropout1_0-3_1.h5
Evaluating model vgg16_dropout1_0-3_1.h5
Loading model vgg16_dropout1_0-4_1.h5
Evaluating model vgg16_dropout1_0-4_1.h5
Loading model vgg16_dropout1_0-5_1.h5
Evaluating model vgg16_dropout1_0-5_1.h5
Loading model vgg16_dropout1_0-6_1.h5
Evaluating model vgg16_dropout1_0-6_1.h5
Loading model vgg16_dropout1_0-7_1.h5
Evaluating model vgg16_dropout1_0-7_1.h5
Loading model vgg16_dropout1_0-8_1.h5
Evaluating model vgg16_dropout1_0-8_1.h5
Evaluation complete.



In [4]:
# create table of evaluation results
table = c2t.eval_table(model_metrics, 'Dropout1', decimals=3)

In [5]:
table

,acc,loss,mpcr
Dropout 1,,,
0,0.923,0.201,0.628
0.1,0.928,0.228,0.623
0.2,0.662,5.448,0.200
0.3,0.662,5.448,0.200
0.4,0.662,5.448,0.200
0.5,0.912,0.250,0.541
0.6,0.906,0.279,0.494
0.7,0.904,0.288,0.498
0.8,0.862,0.366,0.386


In [6]:
# run garbage collection after execution
gc.collect()

142880